In [1]:
import os
import sys

current_dir = os.getcwd()
parent_parent_dir = os.path.abspath(os.path.join(current_dir, "../eval", ".."))
sys.path.append(parent_parent_dir)

from state_control.controllable_rwkv import VectorControlRWKV, ContrastControlRWKV
import ipywidgets as widgets
from IPython.display import display

Using /root/.cache/torch_extensions/py38_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py38_cu118/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module wkv_cuda...


ninja: no work to do.


In [2]:
MODEL_PATH = r'../../models/RWKV-5-World-1B5-v2-20231025-ctx4096'

In [3]:
model = VectorControlRWKV(MODEL_PATH)

RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 6

Loading ../../models/RWKV-5-World-1B5-v2-20231025-ctx4096.pth ...
Strategy: (total 24+1=25 layers)
* cuda [float16, float16], store 25 layers
0-cuda-float16-float16 1-cuda-float16-float16 2-cuda-float16-float16 3-cuda-float16-float16 4-cuda-float16-float16 5-cuda-float16-float16 6-cuda-float16-float16 7-cuda-float16-float16 8-cuda-float16-float16 9-cuda-float16-float16 10-cuda-float16-float16 11-cuda-float16-float16 12-cuda-float16-float16 13-cuda-float16-float16 14-cuda-float16-float16 15-cuda-float16-float16 16-cuda-float16-float16 17-cuda-float16-float16 18-cuda-float16-float16 19-cuda-float16-float16 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 
emb.weight                        f16      cpu  65536  2048 
blocks.0.ln1.weight               f16   cuda:0   2048       
blocks.0.ln1.bias                 f16   cuda:0   2048       
blocks.0.ln2.weight               f16 

Using /root/.cache/torch_extensions/py38_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py38_cu118/rwkv5/build.ninja...
Building extension module rwkv5...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module rwkv5...


ninja: no work to do.


### Language Control Demo
##### Here, we demonstrate the model's capability to control the generated language (Chinese/English) using directional vectors.

First, we construct pairs of prompt words that have the same meaning but are in different languages, providing the model with directions towards various languages.

In [4]:
pos_str_list = ['这是一个句子', '这句话可以是没有任何意义的']
neg_str_list = ['Here is a sentence', 'This sentence can be meaningless.']

Tokenize the sentences and then update the control vectors.

In [5]:
pos_ids_list = [model.tokenizer.encode(x) for x in pos_str_list]
neg_ids_list = [model.tokenizer.encode(x) for x in neg_str_list]

model.update(pos_ids_list, neg_ids_list)

/root/miniconda3/lib/python3.8/site-packages/rwkv/model.py:971: UserWarning: operator() profile_node %187 : int = prim::profile_ivalue(%output_dtype.7)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  x, state[offset] = FFN(


We create a slider to easily adjust the value of the control factor.
You can adjust the slider to see how the control effect changes.

In [6]:
language_control_slider = widgets.FloatSlider(
    value=0,
    min=-1.2,
    max=1.5,
    step=0.01,
    description='Coeff:',
    orientation='horizontal',
)

display(language_control_slider)

FloatSlider(value=0.0, description='Coeff:', max=1.5, min=-1.2, step=0.01)

In [7]:
test_input = "\nHere is a story:"

input_ids = model.tokenizer.encode(test_input)
output_ids = model.generate(input_ids, control_coeff=language_control_slider.value, max_length=50, top_k=1)
print(model.tokenizer.decode(output_ids))


A man was walking down the street when he saw a woman crying. He asked her what was wrong, and she said she had just lost her job. The man felt sorry for her and asked if he could help her in any way. The


You can try both from English to Chinese and from Chinese to English

### output format control
##### Next, we will show how to manipulate the model's output format to output content in Python list.

In [8]:
neg_str_list = [
    "The cat sat on the mat.",
    "It is raining heavily outside.",
    "Learning new languages can be fun and challenging.",
    "Space exploration offers great opportunities for scientific advancement.",
    "Healthy eating contributes to overall wellness."
]

pos_str_list = [
    "['The', 'cat', 'sat', 'on', 'the', 'mat.']",
    "['It', 'is', 'raining', 'heavily', 'outside.']",
    "['Learning', 'new', 'languages', 'can', 'be', 'fun', 'and', 'challenging.']",
    "['Space', 'exploration', 'offers', 'great', 'opportunities', 'for', 'scientific', 'advancement.']",
    "['Healthy', 'eating', 'contributes', 'to', 'overall', 'wellness.']"
]

In [9]:
pos_ids_list = [model.tokenizer.encode(x) for x in pos_str_list]
neg_ids_list = [model.tokenizer.encode(x) for x in neg_str_list]

model.update(pos_ids_list, neg_ids_list)

In [10]:
control_slider = widgets.FloatSlider(
    value=0,
    min=0.,
    max=2.0,
    step=0.01,
    description='Coeff:',
    orientation='horizontal',
)

display(control_slider)

FloatSlider(value=0.0, description='Coeff:', max=2.0, step=0.01)

Adjust the slider to see how the control effect changes.

In [11]:
test_input = "\nUser: List 10 well-known physicists.\n\nAssistant:"

input_ids = model.tokenizer.encode(test_input)
output_ids = model.generate(input_ids, control_coeff=control_slider.value, max_length=50, top_k=1)
print(model.tokenizer.decode(output_ids))

 1. Albert Einstein
2. Isaac Newton
3. Galileo Galilei
4. James Clerk Maxwell
5. Louis Pasteur
6. Charles Darwin
7. Marie Curie
8. Albert Einstein
9. Richard Feyn


### Control of Emotion in Model Output
##### Next, we show how we control the emotions in the model.

In [12]:
pos_str_list = [
    "Question: How was your day?\n\nAnswer: It was fantastic, full of exciting opportunities!",
    "Question: Did you enjoy your meal?\n\nAnswer: Absolutely, it was delicious and satisfying!",
    "Question: Are you looking forward to the weekend?\n\nAnswer: Definitely, I have so many fun plans!",
    "Question: How do you feel about your job?\n\nAnswer: I love it, it's challenging and rewarding!",
    "Question: Did you like the movie?\n\nAnswer: Yes, it was entertaining and thought-provoking!",
    "Question: How's the weather today?\n\nAnswer: It's lovely, sunny and warm!",
    "Question: Are you happy with your new phone?\n\nAnswer: Absolutely, it has amazing features!",
    "Question: Did you enjoy your vacation?\n\nAnswer: Yes, it was relaxing and fun!",
    "Question: How do you feel about the future?\n\nAnswer: I'm optimistic, there are so many possibilities!",
    "Question: Are you satisfied with your current living situation?\n\nAnswer: Yes, I'm very comfortable and content!"
]

neg_str_list = [
    "Question: How was your day?\n\nAnswer: It was terrible, nothing went right.",
    "Question: Did you enjoy your meal?\n\nAnswer: Not at all, it was bland and unappetizing.",
    "Question: Are you looking forward to the weekend?\n\nAnswer: Not really, I have nothing interesting to do.",
    "Question: How do you feel about your job?\n\nAnswer: I dislike it, it's stressful and unrewarding.",
    "Question: Did you like the movie?\n\nAnswer: No, it was boring and poorly made.",
    "Question: How's the weather today?\n\nAnswer: It's dreadful, cold and rainy.",
    "Question: Are you happy with your new phone?\n\nAnswer: Not at all, it's complicated and glitchy.",
    "Question: Did you enjoy your vacation?\n\nAnswer: No, it was stressful and unenjoyable.",
    "Question: How do you feel about the future?\n\nAnswer: I'm pessimistic, things seem uncertain and scary.",
    "Question: Are you satisfied with your current living situation?\n\nAnswer: No, I'm unhappy and looking for a change."
]

In [13]:
pos_ids_list = [model.tokenizer.encode(x) for x in pos_str_list]
neg_ids_list = [model.tokenizer.encode(x) for x in neg_str_list]

model.update(pos_ids_list, neg_ids_list)

In [14]:
control_slider = widgets.FloatSlider(
    value=0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description='Coeff:',
    orientation='horizontal',
)

display(control_slider)

FloatSlider(value=0.0, description='Coeff:', max=1.0, min=-1.0, step=0.01)

Adjust the slider to see how the control effect changes.

In [15]:
test_input = "\nMy mood"

input_ids = model.tokenizer.encode(test_input)
output_ids = model.generate(input_ids, control_coeff=control_slider.value, max_length=50, top_k=1)
print(model.tokenizer.decode(output_ids))

 is so much better now. I'm not depressed anymore. I'm not sad anymore. I'm not angry anymore. I'm not frustrated anymore. I'm not angry at anyone. I'm not angry at myself. I'm not angry at my


In [16]:
# 实验部分

In [27]:
import pickle
# 从文件加载字符串列表
def load_string_list(filename):
    with open(filename, 'rb') as file:
        string_list = pickle.load(file)
    return string_list
def save_data(string_list, filename):
    with open(filename, 'wb') as file:
        pickle.dump(string_list, file)

In [18]:
lambada_data = load_string_list('lambada.pkl')

In [ ]:
for factor in [0., 0.5, 1.0, 1.5, 2.0, -0.5, -1.0, -1.5, -2.0]:
    generated_data = []
    for idx, sample in enumerate(lambada_data):
        input_ids = model.tokenizer.encode(sample)
        input_ids = input_ids[:32]
        input_str = model.tokenizer.decode(input_ids)
        # print(input_str)
        output_ids = model.generate(input_ids, control_coeff=factor, max_length=32, top_k=1)
        output_str = model.tokenizer.decode(output_ids)
        generated_data.append((input_str, output_str))
        print(idx, (input_str, output_str))
        print('-' * 50)
    save_data(generated_data, f'emotion_{factor}.pkl')

0 ('In my palm is a clear stone, and inside it is a small ivory statuette. A guardian angel.\n\n"Figured if you\'re going to', ' be a guardian angel, you might as well be a guardian angel of the right kind."\n\n"I\'m not sure I understand."\n\n"You\'re not')
--------------------------------------------------
1 ('Give me a minute to change and I\'ll meet you at the docks." She\'d forced those words through her teeth.\n\n"No need to change.', ' I\'ll be there in a few minutes."\n\n"I\'ll be waiting."\n\nShe\'d been waiting for a few minutes when he finally arrived. He\'d')
--------------------------------------------------
2 ('"Only one source I know of that would be likely to cough up enough money to finance a phony sleep research facility and pay people big bucks to solve crimes', '," said the man. "And that\'s the FBI."\n\n"I don\'t know about that," said the woman. "I think it\'s more likely')
--------------------------------------------------
3 ("Helen's heart broke a little in the 